In [8]:

def p_start(p):
    """
    start : INT MAIN LPAREN RPAREN LBRACE statement_list RBRACE
    """
    p[0] = p[6]


def p_for(p):
    """
    for :  FOR LPAREN check_for new_scope statement gen_new_label cond SEMI unary RPAREN LBRACE gen_new_label statement_list cond_label RBRACE uncheck_for delete_scope
    """
    p[0] = ("for", p[3], p[5], p[7], p[9], p[10], p[11])

    Tables[-1].name = "FOR"


def p_check_for(p):
    """
    check_for : empty
    """
    global for_expr
    for_expr = True


def p_uncheck_for(p):
    """
    uncheck_for : empty
    """
    global for_expr
    for_expr = False
    utils.print_label()


def p_cond_label(p):
    """
    cond_label : empty
    """
    utils.goto(label_stack[-1] + str(label_stack_no[-1] - 2))


def p_new_scope(p):
    """
    new_scope : empty
    """
    Stack.append(ScopeTable())
    label_stack.append(label_stack[-1] + str(label_stack_no[-1]) + "_")
    label_stack_no.append(0)

    idm.modify("push")


def p_delete_scope(p):
    """
    delete_scope : empty
    """

    Tables.append(Stack.pop())
    label_stack.pop()
    label_stack_no.pop()

    idm.modify("pop")


def p_new_tab(p):
    """
    new_tab : empty
    """
    p[0] = p[1]
    print("\t", end="")


def p_gen_new_label(p):
    """
    gen_new_label : empty
    """
    p[0] = p[1]
    utils.print_label()


def p_switch(p):
    """
    switch : SWITCH LPAREN new_scope switch_expr RPAREN LBRACE labeled_statement_list RBRACE delete_scope
    """
    p[0] = ("switch", p[3], p[5], p[6], p[7])
    Tables[-1].name = "SWICTH"
    global code_list
    code_list.append("l_comparisons:")
    # cprint("l_comparisons:", "cyan")
    # print cases here
    for case in cases_list:
        if case[0] == "case":
            try:
                temp = idm.dikt[switch_expr]["scope"][-1] - 1
                code_list.append(f"\tEQ\t{switch_expr}_{temp}\t{case[1]}\t{case[2]}")
                # cprint(f"\tEQ\t{switch_expr}_{temp}\t{case[1]}\t{case[2]}", "cyan")
            except:
                code_list.append(f"\tEQ\t{switch_expr}\t{case[1]}\t{case[2]}")
                # cprint(f"\tEQ\t{switch_expr}\t{case[1]}\t{case[2]}", "cyan")
        else:
            utils.goto(f"{case[1]}")
    code_list.append("l_next_switch:")
    # cprint("l_next_switch:", "cyan")


def p_switch_expr(p):
    """
    switch_expr : ID
                    | ICONST
    """
    p[0] = p[1]

    global switch_expr
    if utils.isDeclared(p[1][0]):
        switch_expr = p[1][0]
    else:
        utils.printNotDeclared(f"{p[1][0]}")

    utils.goto("l_comparisons")


def p_labeled_statement_list(p):
    """
    labeled_statement_list : labeled_statement labeled_statement_list
    """
    p[0] = (p[1], p[2])


def p_labeled_statement_list_1(p):
    """
    labeled_statement_list : empty
    """
    p[0] = p[1]


def p_labeled_statement(p):
    """
    labeled_statement : CASE gen_new_label const_expr COLON new_scope statement_list delete_scope
    """
    p[0] = ("case", p[2], p[5])
    Tables[-1].name = "SWITCH CASE"
    temp = label_stack_no[-1] - 1
    cases_list.append(("case", p[3], f"{label_stack[-1]}{temp}"))


def p_labeled_statement_1(p):
    """
    labeled_statement :  DEFAULT COLON gen_new_label new_scope statement_list delete_scope
    """
    p[0] = ("default", p[5])
    Tables[-1].name = "SWITCH DEF"
    temp = label_stack_no[-1] - 1
    cases_list.append(("default", f"{label_stack[-1]}{temp}"))
    utils.goto("l_next_switch")


def p_labeled_statement_2(p):
    """
    labeled_statement :  labeled_statement BREAK SEMI
    """
    p[0] = ("labeled_statement", p[1], p[3])
    utils.goto("l_next_switch")


def p_const_expr(p):
    """
    const_expr : ICONST
                            | CCONST
    """
    p[0] = p[1]


def p_statement_list(p):
    """
    statement_list : statement statement_list
    """
    p[0] = (p[1], p[2])  # Doubtful about this


def p_statement_list_1(p):
    """
    statement_list : empty
    """
    p[0] = p[1]


def p_statement(p):
    """
    statement : unary
                            | assign
                            | declaration
                            | for
                            | switch
    """
    p[0] = p[1]


def p_empty(p):
    "empty :"
    ...


def p_assign(p):
    """
    assign : ID EQUALS expr SEMI
    """
    p[0] = ("EXPRESSION CALCULATED", "=", p[1], p[3])

    if not utils.isDeclared(p[1]):
        utils.printNotDeclared(p[1])

    if type(p[3]) == tuple:
        utils.assign(p[1], p[3][0])
        Stack[-1].update_val(p[1], utils.getActualValue(p[3][0]))
    elif p[3] not in idm.dikt:
        utils.assign(p[1], "t" + str(p[3]))
    else:
        utils.assign(p[1], p[3])


def p_assign_1(p):
    """
    assign : ID EQUALS CCONST SEMI
    """
    p[0] = (p[1], p[3])

    if not utils.isDeclared(p[1]):
        utils.printNotDeclared(p[1])
    else:
        utils.assign(p[1], p[3])
        Stack[-1].update_val(p[1], utils.getActualValue(p[3]))


def p_cond(p):
    """
    cond : ID LT ICONST
            | ID LE ICONST
            | ID GE ICONST
            | ID GT ICONST
            | ID LT ID
            | ID LE ID
            | ID GE ID
            | ID GT ID
            | ID NE ICONST
            | ID NE ID
            | ID NE FCONST
            | ICONST NE ID
            | ICONST NE ICONST
            | ID LE FCONST
            | ID GE FCONST
            | ID GT FCONST
            | ID LT FCONST
            | ICONST LE ICONST
            | ICONST GE ICONST
            | ICONST GT ICONST
            | ICONST LT ICONST
            | FCONST LE FCONST
            | FCONST GE FCONST
            | FCONST GT FCONST
            | FCONST LT FCONST
            | ID EQ ID
            | ID EQ ICONST
            | ID EQ FCONST
            | ICONST EQ ID
            | FCONST EQ ID
            | ICONST EQ ICONST
            | FCONST EQ FCONST
    """
    sym_map = {"<=": "LE", "<": "LT", ">=": "GE", ">": "GT", "==": "EQ", "!=": "NE"}
    if p[2] == "<=":
        p[0] = ("CONDI", "<=", p[1], p[3])
    elif p[2] == "<":
        p[0] = ("CONDI", "<", p[1], p[3])
    elif p[2] == ">=":
        p[0] = ("CONDI", ">=", p[1], p[3])
    elif p[2] == ">":
        p[0] = ("CONDI", ">", p[1], p[3])
    elif p[2] == "==":
        p[0] = ("CONDI", "==", p[1], p[3])
    elif p[2] == "!=":
        p[0] = ("CONDI", "!=", p[1], p[3])
    global code_list

    if p[1] in idm.dikt:
        if utils.isDeclared(p[1]):
            temp = idm.dikt[p[1]]["scope"][-1] - 1
            t1 = f"{p[1]}_{temp}"
        else:
            temp = "\b" * 8
            utils.printNotDeclared(f"{p[1]}")
    else:
        t1 = p[1]

    if p[3] in idm.dikt:
        if utils.isDeclared(p[3]):
            temp = idm.dikt[p[3]]["scope"][-1] - 1
            t2 = f"{p[3]}_{temp}"
        else:
            temp = "\b" * 8
            code_list.append(f"{temp}")
            # cprint(f"{temp}", "cyan")
            utils.printNotDeclared(f"{p[3]}")
    else:
        t2 = p[3]

    try:
        global for_expr
        if for_expr == False:
            code_list.append(
                "\t%s\t%s\t%s\t%s"
                % (sym_map[p[2]], t1, t2, label_stack[-1] + str(label_stack_no[-1]))
            )
            # cprint(
            #     "\t%s\t%s\t%s\t%s"
            #     % (sym_map[p[2]], t1, t2, label_stack[-1] + str(label_stack_no[-1])),
            #     "cyan",
            # )
            utils.goto(label_stack[-1] + str(label_stack_no[-1] + 1))
            code_list.append("%s:" % (label_stack[-1] + str(label_stack_no[-1])))
            # cprint("%s:" % (label_stack[-1] + str(label_stack_no[-1])), "cyan")
            label_stack_no[-1] += 1
        elif for_expr == True:
            code_list.append(
                "\t%s\t%s\t%s\t%s"
                % (
                    sym_map[p[2]],
                    t1,
                    t2,
                    label_stack[-1] + str(label_stack_no[-1] + 1),
                )
            )
            # cprint(
            #     "\t%s\t%s\t%s\t%s"
            #     % (
            #         sym_map[p[2]],
            #         t1,
            #         t2,
            #         label_stack[-1] + str(label_stack_no[-1] + 1),
            #     ),
            #     "cyan",
            # )
            utils.goto(label_stack[-1] + str(label_stack_no[-1] + 2))
            code_list.append("%s:" % (label_stack[-1] + str(label_stack_no[-1])))
            # cprint("%s:" % (label_stack[-1] + str(label_stack_no[-1])), "cyan")
            label_stack_no[-1] += 1
    except:
        ...


def p_cond_1(p):
    """
    cond : ID
    """
    p[0] = p[1]
    global code_list
    if p[1] in idm.dikt:
        if utils.isDeclared(p[1]):
            temp = idm.dikt[p[1]]["scope"][-1] - 1
            t1 = f"{p[1]}_{temp}"
        else:
            temp = "\b" * 8
            code_list.append(f"{temp}")
            # cprint(f"{temp}", "cyan")
            utils.printNotDeclared(f"{p[1]}")
    else:
        t1 = p[1]

    try:
        code_list.append("GT\t%s\t0\tl%s" % (t1, label_stack_no[-1]))
        # cprint("GT\t%s\t0\tl%s" % (t1, label_stack_no[-1]), "cyan")
        utils.goto(label_stack[-1] + str(label_stack_no[-1] + 1))
        utils.print_label()
    except:
        ...


def p_unary_pre(p):
    """
    unary : PLUSPLUS ID
       | MINUSMINUS ID
    """
    global code_list
    if p[1] == "++":
        p[0] = ("PREINC", "++", p[2])
        code_list.append(f"\PREINC\t\t\t{p[2]}")
        # cprint(f"\PREINC\t\t\t{p[2]}", "cyan")
    elif p[1] == "--":
        p[0] = ("PREDEC", "--", p[2])
        code_list.append(f"\PREDEC\t\t\t{p[2]}")
        # cprint(f"\PREDEC\t\t\t{p[2]}", "cyan")
    utils.goto(label_stack[-1] + str(label_stack_no[-1] - 2))


def p_unary_post(p):
    """
    unary : ID PLUSPLUS
       | ID MINUSMINUS
    """
    global code_list
    if p[2] == "++":
        p[0] = ("POSTINC", "++", p[1])
        code_list.append(f"\tPOSTINC\t\t\t{p[1]}")
        # cprint(f"\tPOSTINC\t\t\t{p[1]}", "cyan")
    elif p[2] == "--":
        p[0] = ("POSTDEC", "--", p[1])
        code_list.append(f"\tPOSTDEC\t\t\t{p[1]}")
        # cprint(f"\tPOSTDEC\t\t\t{p[1]}", "cyan")
    utils.goto(label_stack[-1] + str(label_stack_no[-1] - 2))


def p_declaration(p):
    """
    declaration : types vee SEMI
                            | types arr SEMI
    """
    p[0] = (p[2], p[1])
    # Update the types of the above declared variables
    Stack[-1].update_type(p[1])


def p_types(p):
    """
    types : INT
            | FLOAT
            | DOUBLE
            | CHAR
            | LONG
            | REGISTER
    """
    p[0] = p[1]


def p_vee_1(p):
    """
    vee : vee COMMA vee
    """
    p[0] = (p[1], ",", p[3])


def p_vee_2(p):
    """
    vee : ID
    """
    p[0] = p[1]

    try:
        Stack[-1].insert(p[1], "")
        utils.decl_var(p[1])

    except Exception:
        ...


def p_vee_3(p):
    """
    vee : init
    """
    p[0] = p[1]


def p_init(p):
    """
    init : ID EQUALS expr
    """
    p[0] = ("EXPRESSSION CALCULATED", "=", p[1], p[3])

    try:
        Stack[-1].insert(p[1], "")
        utils.decl_var(p[1])
        if type(p[3]) == tuple:
            utils.assign(p[1], p[3][0])
            Stack[-1].update_val(p[1], utils.getActualValue(p[3][0]))
        elif p[3] not in idm.dikt:
            utils.assign(p[1], "t" + str(p[3]))
        else:
            if utils.isDeclared(p[3]):
                utils.assign(p[1], p[3])
            else:
                utils.printNotDeclared(p[3])
    except Exception:
        wprint("Caught exception")


def p_init_1(p):
    """
    init : ID EQUALS CCONST
    """
    p[0] = ("=", p[1], p[3])

    try:
        Stack[-1].insert(p[1], "")
        Stack[-1].update_val(p[1], utils.getActualValue(p[3]))
        utils.decl_var(p[1])
        utils.assign(p[1], p[3])
    except Exception:
        ...


def p_arr(p):
    """
    arr : ID open_bracket
    """
    p[0] = p[1]
    try:
        Stack[-1].insert(p[1], "")

        utils.decl_var(p[1])
    except:
        ...


def p_open_bracket(p):
    """
    open_bracket : LBRACKET ICONST RBRACKET
                            | LBRACKET ICONST RBRACKET open_bracket
    """

    try:
        p[0] = (p[1], p[2], p[3], p[4])
    except:
        p[0] = (p[1], p[2], p[3])


def p_expr(p):
    """
    expr : expr PLUS term
            | expr MINUS term
    """
    global temp_var_no
    global code_list
    if type(p[1]) == type((1,)):  # ICONST or FCONST
        t1 = utils.decl_temp(p[1][0], p[1][1])
        t1 = f"t{t1}"
    elif p[1] in idm.dikt:  # ID
        if utils.isDeclared(p[1]):
            temp = idm.dikt[p[1]]["scope"][-1] - 1
            t1 = f"{p[1]}_{temp}"
        else:
            utils.printNotDeclared(p[1])
    else:
        t1 = f"t{p[1]}"

    if type(p[3]) == tuple:
        t2 = utils.decl_temp(p[3][0], p[3][1])
        t2 = f"t{t2}"
    elif p[3] in idm.dikt:
        if utils.isDeclared(p[3]):
            temp = idm.dikt[p[3]]["scope"][-1] - 1
            t2 = f"{p[3]}_{temp}"
        else:
            utils.printNotDeclared(p[3])
    else:
        t2 = f"t{p[3]}"

    try:

        if p[2] == "+":
            utils.decl_var(f"t{temp_var_no}")
            code_list.append(f"\tADD\t{t1}\t{t2}\tt{temp_var_no}")
            # cprint(f"\tADD\t{t1}\t{t2}\tt{temp_var_no}", "cyan")
            temp_var_no += 1
        elif p[2] == "-":
            utils.decl_var(f"t{temp_var_no}")
            code_list.append(f"\tSUB\t{t1}\t{t2}\tt{temp_var_no}")
            # cprint(f"\tSUB\t{t1}\t{t2}\tt{temp_var_no}", "cyan")
            temp_var_no += 1
    except Exception:
        ...

    p[0] = str(temp_var_no - 1)


def p_expr_1(p):
    """
    expr : term
    """
    p[0] = p[1]


def p_term(p):
    """
    term : term TIMES factor
            | term DIVIDE factor
    """
    global temp_var_no
    global code_list
    p[0] = (p[1], p[3])
    if type(p[1]) == type((1,)):  # ICONST or FCONST
        t1 = utils.decl_temp(p[1][0], p[1][1])
        t1 = f"t{t1}"
    elif p[1] in idm.dikt:  # ID
        temp = idm.dikt[p[1]]["scope"][-1]
        t1 = f"{p[1]}_{temp}"
    else:
        t1 = f"t{p[1]}"
    # print(p[1], p[2], p[3])
    if type(p[3]) == tuple:
        t2 = utils.decl_temp(p[3][0], p[3][1])
        t2 = f"t{t2}"
    elif p[3] in idm.dikt:
        temp = idm.dikt[p[3]]["scope"][-1]
        t2 = f"{p[3]}_{temp}"
    else:
        t2 = f"t{p[3]}"

    if p[2] == "*":
        utils.decl_var(f"t{temp_var_no}")
        code_list.append(f"\tMUL\t{t1}\t{t2}\tt{temp_var_no}")
        # cprint(f"\tMUL\t{t1}\t{t2}\tt{temp_var_no}", "cyan")
        temp_var_no += 1

    elif p[2] == "/":
        utils.decl_var(f"t{temp_var_no}")
        code_list.append(f"\tDIV\t{t1}\t{t2}\tt{temp_var_no}")
        # cprint(f"\tDIV\t{t1}\t{t2}\tt{temp_var_no}", "cyan")
        temp_var_no += 1

    p[0] = str(temp_var_no - 1)


def p_term_1(p):
    """
    term : factor
    """
    p[0] = p[1]


def p_factor(p):
    """
    factor : ID
    """
    p[0] = p[1]


def p_factor_1(p):
    """
    factor : ICONST
    """
    p[0] = (p[1], "INT")


def p_factor_2(p):
    """
    factor : FCONST
    """
    p[0] = (p[1], "FLOAT")



In [27]:
x = [j.__doc__ for i,j in globals().items() if i.startswith("p_")]
with open("grammer.txt", 'w') as f:
    for i in x:
        f.write(i.lstrip())

In [26]:
for i in range(10,2, -1):
    print(i)

10
9
8
7
6
5
4
3
